In [1]:
from pyiron import Project
import pandas as pd

In [2]:
pr = Project("AlLiMEAM")

Using openkim potentials requires a kim init command, which has to be placed before the structure. This is not directly possible in pyiron due to the order in the control file.

A somewhat hacky workaround is to place the read_data command into the potential file and delete it from the original control.inp file.

In [3]:
s = pr.create.structure.ase.bulk("Al", cubic=True)
s[1] = "Li"
job = pr.create.job.Lammps("PotDF", delete_existing_job=True)
job.structure = s

pot = pd.DataFrame(
    {
        "Name": "KIM MEAM",
        "Filename": [[]],
        "Model": ["Custom"],
        "Species": [["Al", "Li"]],
        "Config": [
            [
                'kim init MEAM_LAMMPS_RoyDuttaChakraborti_2021_AlLi__MO_971738391444_000 metal\n',
                'read_data structure.inp\n', # This is a really dirty fix for the kim init problem
                'kim interactions Al Li\n',
            ]
        ],
    }
)

job.potential = pot
del job.input.control["read_data"]
job.input.control

,Parameter,Value,Comment
0,units,metal,
1,dimension,3,
2,boundary,p p p,
3,atom_style,atomic,
4,include,potential.inp,
5,fix___ensemble,all nve,
6,variable___dumptime,equal 100,
7,variable___thermotime,equal 100,
8,dump___1,all custom ${dumptime} dump.out id type xsu ys...,
9,dump_modify___1,"sort id format line ""%d %d %20.15g %20.15g %20...",


In [4]:
job.run()

The job PotDF was saved and received the ID: 648448
